In [138]:
############################
## import packages
############################
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy

print('pandas version:', pd.__version__)
print('numpy version:', np.__version__)
print('matplotlib version:', matplotlib.__version__)
print('seaborn version:', sns.__version__)
print('sklearn version:', sklearn.__version__)
print('scipy version:', scipy.__version__)

pandas version: 1.1.0
numpy version: 1.19.1
matplotlib version: 3.3.0
seaborn version: 0.10.1
sklearn version: 0.23.2
scipy version: 1.5.0


In [139]:
############################
## Get data
############################
import os

# data location and path
DATA_URL = 'https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz'
DATA_FOLDER = 'C:\\Users\\Admin\\Documents\\Data Analysis\\Data\\Housing data'
DATA_FILE = 'housing.tgz'

# function to download data
def get_data(url=DATA_URL, path=DATA_FOLDER, file=DATA_FILE):
    """Downloads & Extracts the contents of `DATA_URL` into `DATA_FOLDER`
    # Arguments:
        url, string: the download link
        path, string: where to download & extract data
        file, string: downloaded filename
    """
    import tarfile
    import urllib
    if not os.path.exists(path):                        # create data folder if absent
        os.makedirs(path)
    tgz_path = os.path.join(path, file)
    if not os.path.exists(tgz_path):                    # download data file if absent
        urllib.request.urlretrieve(url=url, filename=tgz_path)
    file_tgz = tarfile.open(name=tgz_path)              # extract tar-zipped file
    file_tgz.extractall(path)
    file_tgz.close()

# function to load data
FILE = 'housing.csv'
def load_data(path=DATA_FOLDER, file=FILE):
    """Loads data into a pandas dataframe.
    # Arguments:
        path, string: the path where data lives
        file, string: extacted filename
    # Returns:
        data, pd.DataFrame: data as a pandas dataframe
    """
    get_data()
    csv_path = os.path.join(path, file)
    return pd.read_csv(csv_path)

In [140]:
##########################
# load and inspect data
##########################
housing = load_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [141]:
housing.info()                  # reveals null counts if any ('total_bedrooms')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [142]:
housing.nunique()   # reveals hidden categorical features (numeric type but with low unique counts)

longitude               844
latitude                862
housing_median_age       52
total_rooms            5926
total_bedrooms         1923
population             3888
households             1815
median_income         12928
median_house_value     3842
ocean_proximity           5
dtype: int64

In [143]:
housing['ocean_proximity'].value_counts()       # category frequencies in 'ocean_proximity'

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [144]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [145]:
"""
#####################################
### quick look and feel
#####################################
# histogram of all numerical features
housing.hist(bins=50, figsize=(15,10))
plt.show()
# -> spike at the right edge of house age and price distributions => data capped ->
# remove those districts from training set (to prevent model from being evaluated poorly if it
# predicts values beyond cap)
# -> many distributions are right-tail-heavy => hard for model to detect patterns ->
# data may need transforming to generate more bell-shaped distributions
# -> features have different scales => data normalization is needed
"""

'\n#####################################\n### quick look and feel\n#####################################\n# histogram of all numerical features\nhousing.hist(bins=50, figsize=(15,10))\nplt.show()\n# -> spike at the right edge of house age and price distributions => data capped ->\n# remove those districts from training set (to prevent model from being evaluated poorly if it\n# predicts values beyond cap)\n# -> many distributions are right-tail-heavy => hard for model to detect patterns ->\n# data may need transforming to generate more bell-shaped distributions\n# -> features have different scales => data normalization is needed\n'

In [146]:
"""
# pairwise scatterplots of numerical features
from scipy.stats import pearsonr
def reg_coef(x, y, label=None, color=None, **kwargs):
    ax = plt.gca()
    r,p = pearsonr(x,y)
    ax.annotate('r = {:.2f}'.format(r), xy=(0.5,0.5), fontsize=14, xycoords='axes fraction', ha='center')
    ax.set_axis_off()

featureSet = ['housing_median_age','total_rooms','population','households','median_income','median_house_value']
sns.set_style('ticks')
g = sns.PairGrid(housing[featureSet], diag_sharey=False, height=1.5, aspect=1.4)
g.map_diag(sns.kdeplot, shade=True)
g.map_lower(sns.scatterplot, s=30, edgecolor='w')
#g.map_lower(sns.regplot, scatter_kws={'s':15})
g.map_upper(reg_coef)
plt.show()
"""

"\n# pairwise scatterplots of numerical features\nfrom scipy.stats import pearsonr\ndef reg_coef(x, y, label=None, color=None, **kwargs):\n    ax = plt.gca()\n    r,p = pearsonr(x,y)\n    ax.annotate('r = {:.2f}'.format(r), xy=(0.5,0.5), fontsize=14, xycoords='axes fraction', ha='center')\n    ax.set_axis_off()\n\nfeatureSet = ['housing_median_age','total_rooms','population','households','median_income','median_house_value']\nsns.set_style('ticks')\ng = sns.PairGrid(housing[featureSet], diag_sharey=False, height=1.5, aspect=1.4)\ng.map_diag(sns.kdeplot, shade=True)\ng.map_lower(sns.scatterplot, s=30, edgecolor='w')\n#g.map_lower(sns.regplot, scatter_kws={'s':15})\ng.map_upper(reg_coef)\nplt.show()\n"

In [147]:
"""
# zooming in income-house price scatterplot
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.5, figsize=(10,7))
plt.show()
# plot shows cap at 500000, also few horizontal lines at ~450000, 350000, 280000 etc ->
# these data points should be removed to prevent model from learning to predict similar quirks
"""

'\n# zooming in income-house price scatterplot\nhousing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.5, figsize=(10,7))\nplt.show()\n# plot shows cap at 500000, also few horizontal lines at ~450000, 350000, 280000 etc ->\n# these data points should be removed to prevent model from learning to predict similar quirks\n'

In [148]:
##############################################
### stratified sampling
##############################################
# because price is related to income, strata (hidden clusters) in income distribution will be used
# to generate stratified training and test sets ->
# use pd.cut() to create a discrete set of income strata [1, 2, 3, 4, 5] and merge all
# incomes above 5 into stratum 5
housing['income_cat'] = pd.cut(x=housing['median_income'], bins=[0, 1.5, 3, 4.5, 6, np.inf],                                      labels=[1, 2, 3, 4, 5])
housing['income_cat'].value_counts(normalize=True)

3    0.350581
2    0.318847
4    0.176308
5    0.114438
1    0.039826
Name: income_cat, dtype: float64

In [149]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X=housing, y=housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
# both train and test sets have same income strata proportions as the original data

In [150]:
strat_train_set['income_cat'].value_counts(normalize=True)

3    0.350594
2    0.318859
4    0.176296
5    0.114402
1    0.039850
Name: income_cat, dtype: float64

In [151]:
strat_test_set['income_cat'].value_counts(normalize=True)

3    0.350533
2    0.318798
4    0.176357
5    0.114583
1    0.039729
Name: income_cat, dtype: float64

In [152]:
# next remove unneeded 'income_cat' column both sets
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)

strat_train_set.shape, strat_test_set.shape

((16512, 10), (4128, 10))

In [153]:
#####################################
### visualize training data set
#####################################
# keep away test data and explore only training dataset (use a copy to prevent data corrpuption)
# (install pyarrow with <conda install pyarrow> if needed)
strat_test_set.reset_index().to_feather(fname='strat_test_set.f')
housing = strat_train_set.copy()

In [154]:
"""
# scatterplot of geographical distribution of housing based on latitude and logitude
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1, figsize=(10,7))
plt.show()
# plot shows California coastline, and using alpha=0.1 highlights high-density areas
"""

"\n# scatterplot of geographical distribution of housing based on latitude and logitude\nhousing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1, figsize=(10,7))\nplt.show()\n# plot shows California coastline, and using alpha=0.1 highlights high-density areas\n"

In [155]:
"""
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,
             s=housing['population']/100, label='population', c='median_house_value',
             cmap=plt.get_cmap('jet'), colorbar=True, figsize=(10,7))
plt.legend()
plt.show()
# plot highlights relation between housing price, location (ocean proximity) and density
"""

"\nhousing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,\n             s=housing['population']/100, label='population', c='median_house_value',\n             cmap=plt.get_cmap('jet'), colorbar=True, figsize=(10,7))\nplt.legend()\nplt.show()\n# plot highlights relation between housing price, location (ocean proximity) and density\n"

In [156]:
#####################################
## explore feature combination
#####################################
# -> total rooms in a district is uninformative - room/household is better
# -> total bedrooms uninformative - comparing with toal rooms is better
# -> total population is less information - pop/household is better
housing['rooms_per_household'] = housing['total_rooms']/housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

# verify with correlation matrix
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)
# 'bedrooms_per_room' has stronegr correlation with price than 'total_bedrooms'

median_house_value          1.000000
median_income               0.687160
rooms_per_household         0.146285
total_rooms                 0.135097
housing_median_age          0.114110
households                  0.064506
total_bedrooms              0.047689
population_per_household   -0.021985
population                 -0.026920
longitude                  -0.047432
latitude                   -0.142724
bedrooms_per_room          -0.259984
Name: median_house_value, dtype: float64

In [157]:
########################################
## prep data for ML algorithms
########################################
# separate training dataset between features/predictors and label/target
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()
housing.shape, housing_labels.shape

((16512, 9), (16512,))

In [158]:
# impute missing 'total_bedrooms' data with median
from sklearn.impute import SimpleImputer
housing_num = housing.drop('ocean_proximity', axis=1)   # drop categorical feature from median computation
imputer = SimpleImputer(strategy='median')
imputer.fit(housing_num)            # computes median for ALL numeric features
#housing_num.median().values        # shows median values of all numeric features
X = imputer.transform(housing_num)  # impute ALL missing data with median (output is numpy array)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)   # convert back to dataframe
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042
1,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214
2,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621
3,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839
4,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347


In [159]:
# one-hot encoding to transform categorical 'ocean_proximity' to numerical (binary) variables
from sklearn.preprocessing import OneHotEncoder
housing_cat = housing[['ocean_proximity']]
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat.values)
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [160]:
# Custom Transformer class to add combined attributes to data sets
# (to work seamlessly with sklearn functionalities, such as pipelines, this class
# must implement three methods: 'fit()', 'transform()' and 'fit_transform()')
# -> adding TransformerMixin as a base class gives method 'fit_transform()' for free
# -> adding BaseEstimator as a base class and avoiding args and *kwargs in Constructor
#    gives two extra methods '.get_params()' and '.set_params()'
#    (useful for auto-tuning hyperparameters)
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    """Engineers new features from existing ones: `rooms_per_household`,            
    `population_per_household`, `bedrooms_per_room`
    # Arguments:
        add_bedrooms_per_room, bool: defaults to True. Indicates if we want to add the feature  
        `bedrooms_per_room`.
    """
    def __init__(self, add_bedrooms_per_room=True):         # no args or *kwargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        """returns self"""          # Docstring for fit() method
        return self         # nothing else to do
    def transform(self, X, y=None):
        """returns Numpy array of data with added attributes"""
        rooms_per_household = X[:,rooms_ix] / X[:,household_ix]
        population_per_household = X[:,population_ix] / X[:,household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:,bedrooms_ix] / X[:,rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing.values[0:5]

array([[-121.89, 37.29, 38.0, 1568.0, 351.0, 710.0, 339.0, 2.7042,
        '<1H OCEAN'],
       [-121.93, 37.05, 14.0, 679.0, 108.0, 306.0, 113.0, 6.4214,
        '<1H OCEAN'],
       [-117.2, 32.77, 31.0, 1952.0, 471.0, 936.0, 462.0, 2.8621,
        'NEAR OCEAN'],
       [-119.61, 36.31, 25.0, 1847.0, 371.0, 1460.0, 353.0, 1.8839,
        'INLAND'],
       [-118.59, 34.23, 17.0, 6592.0, 1525.0, 4459.0, 1463.0, 3.0347,
        '<1H OCEAN']], dtype=object)

In [161]:
housing_extra_attribs[0:5]

array([[-121.89, 37.29, 38.0, 1568.0, 351.0, 710.0, 339.0, 2.7042,
        '<1H OCEAN', 4.625368731563422, 2.094395280235988],
       [-121.93, 37.05, 14.0, 679.0, 108.0, 306.0, 113.0, 6.4214,
        '<1H OCEAN', 6.008849557522124, 2.7079646017699117],
       [-117.2, 32.77, 31.0, 1952.0, 471.0, 936.0, 462.0, 2.8621,
        'NEAR OCEAN', 4.225108225108225, 2.0259740259740258],
       [-119.61, 36.31, 25.0, 1847.0, 371.0, 1460.0, 353.0, 1.8839,
        'INLAND', 5.232294617563739, 4.135977337110481],
       [-118.59, 34.23, 17.0, 6592.0, 1525.0, 4459.0, 1463.0, 3.0347,
        '<1H OCEAN', 4.50580997949419, 3.047846889952153]], dtype=object)

In [162]:
##############################################################################################
# Data transformation using pipeline
# Pipeline constructor takes a list of (name,estimator) pairs defining a sequence of steps;
# all but last estimator must be transformers (must have a 'fit_transform()' method).
# Calling pipeline's 'fit()' method calls 'fit_transform()' sequentially on all transformers,
# passing output of each call as parameter to next call, until it reaches final estimator,
# for which it just calls 'fit()' method.
# Pipeline exposes same methods as the last estimator: in this example, StandardScaler is a
# transformer, so pipeline has a 'transform()' method that applies to all transforms to data
# in sequence (it also has a 'fit_transform()' that we could call instead of calling 'fit()'
# and then 'transform()')
##############################################################################################
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# pipeline to transform numeric data
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scalar', StandardScaler())
])

# full pipeline (ColumnTransformer transforms both numeric and categorical columns)
num_attribs = housing_num.columns.tolist()      # numeric feature names
cat_attribs = ['ocean_proximity']               # categorical feature name
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepped = full_pipeline.fit_transform(X = housing)
housing_prepped.shape

(16512, 16)

In [163]:
###################################################################
# Select and train model
###################################################################
# first a linear regression model
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X=housing_prepped, y=housing_labels)

# Example below does NOT work when full_pipeline.fit_transform() is used with 'some_data'.
# some_data has only 5 rows of data -> 'ocean_proximity' has 3 values (instead of 5 in
# the full housing data). This gives different dimensionality to some_data_prepped when created
# with full_pipeline.fit_transform(), which throws error with lin_reg.predict(), because
# lin_reg was fitted with training data 'housing_prepped'. Use full_pipeline.transform() instead
# (this works because full_pipeline transforms some_data without fitting to it first - it uses
# earlier fit with housing data instead).
# fit(), fit_transform(), fit_predict() should only be called with training data, never on
# validation/test/new data (https://github.com/ageron/handson-ml/issues/347#issuecomment-501558848).
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepped = full_pipeline.transform(some_data)
print('Prediction:\t', lin_reg.predict(some_data_prepped))
print('Labels:\t', list(some_labels))

Prediction:	 [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Labels:	 [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [164]:
# evaluate model
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepped)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.19819848923

In [165]:
# next a Decision Tree Regressor model
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X=housing_prepped, y=housing_labels)

housing_predictions = tree_reg.predict(housing_prepped)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse
# model overfits training data - cross validation (below) is the way to go

0.0

In [166]:
# next Random Forest Regressor model (Ensemble Learning - trains many Decision Trees)
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(X=housing_prepped, y=housing_labels)

housing_predictions = forest_reg.predict(housing_prepped)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18692.87600982754

In [167]:
# next a Support Vector Machine Regressor model
# (with linear kernel)
from sklearn.svm import SVR
svr_reg = SVR(kernel='linear')
svr_reg.fit(X=housing_prepped, y=housing_labels)

housing_predictions = svr_reg.predict(housing_prepped)
svr_mse = mean_squared_error(housing_labels, housing_predictions)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

111094.6308539982

In [168]:
# (with rbf kernel)
svr_reg = SVR(kernel='rbf')
svr_reg.fit(X=housing_prepped, y=housing_labels)

housing_predictions = svr_reg.predict(housing_prepped)
svr_mse = mean_squared_error(housing_labels, housing_predictions)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

118580.68301157995

In [169]:
############################################
# cross validation (k-fold)
############################################
# Decision Tree model
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator = tree_reg,
                         X = housing_prepped,
                         y = housing_labels,
                         scoring = 'neg_mean_squared_error',
                         cv = 10        # 10-fold CV
                        )
tree_rmse_scores = np.sqrt(-scores)
# cross-validation expects utility function (greater the better) rather than a cost function
# (lower the better), hence scoring function is opposite of MSE (i.e. negative) -> output scores
# is negated to compute RMSE 

# function to display scores for k-fold CV
def display_scores(scores):
    """Displays the scores, their mean, and the standard deviation.
    
    # Arguments:
        scores, np.array: list of scores given by the cross validation procedure.
    """
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard deviation:', scores.std())

display_scores(tree_rmse_scores)

Scores: [68301.73432572 66411.83830823 73067.99662965 68044.75268659
 71386.18356629 72832.52831767 72070.00510292 71568.07122127
 76271.78301862 69601.9569938 ]
Mean: 70955.68501707369
Standard deviation: 2757.5978311315357


In [170]:
# linear regression model
scores = cross_val_score(estimator = lin_reg,
                         X = housing_prepped,
                         y = housing_labels,
                         scoring = 'neg_mean_squared_error',
                         cv = 10
                        )
lin_rmse_scores = np.sqrt(-scores)
display_scores(lin_rmse_scores)

Scores: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean: 69052.46136345083
Standard deviation: 2731.6740017983493


In [171]:
# Random forest model
scores = cross_val_score(estimator = forest_reg,
                         X = housing_prepped,
                         y = housing_labels,
                         scoring = 'neg_mean_squared_error',
                         cv = 10
                        )
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [49074.49312152 47784.3286393  49692.11740759 52207.66648538
 49500.82478611 53271.7779091  49246.28821604 47846.79675513
 52973.56159263 49868.17461888]
Mean: 50146.60295316947
Standard deviation: 1885.1243998022412


In [172]:
####################################################################
# save all model paramaters/hyperparameters, outputs, scores to file
####################################################################
import joblib
path = 'models'
if not os.path.exists(path):
    os.makedirs(path)
joblib.dump(value=forest_reg, filename=path+'/forest_reg.m')

['models/forest_reg.m']

In [173]:
# to load model from file
forest_reg = joblib.load(filename=path+'/forest_reg.m')

In [174]:
##########################################################
# Fine tuning model with grid search (good for small grid)
# use RandomizedSearchCV for large parameter space
##########################################################
from sklearn.model_selection import GridSearchCV

# For Random Forest model ->
# first set of evaluations in dict#1 uses 'bootstrap':[True] (default)
# 3X4 + 2X3 = 18 combinations of hyperparameter values
# 18 X 5 (for 5-fold CV) = 90 rounds of training
param_grid = [
    {'n_estimators':[3, 10, 30], 'max_features':[2, 4, 6, 8]},
    {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features':[2, 3, 4]}
]
forest_reg = RandomForestRegressor()
forest_grid_search = GridSearchCV(estimator = forest_reg,
                                  param_grid = param_grid,
                                  scoring = 'neg_mean_squared_error',
                                  cv = 5,
                                  return_train_score = True,
                                  n_jobs = -1,      # parallelizing jobs
                                  verbose = 2
                                 )
forest_grid_search.fit(X=housing_prepped, y=housing_labels)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   36.0s finished


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [183]:
best_mse = forest_grid_search.best_score_
best_rmse = np.sqrt(-best_mse)
best_rmse

50081.87192950168

In [175]:
forest_grid_search.best_estimator_         # best model

RandomForestRegressor(max_features=8, n_estimators=30)

In [176]:
# evaluation scores for all 18 parameter combinations
cvres = forest_grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

63382.430727482824 {'max_features': 2, 'n_estimators': 3}
55271.40238097063 {'max_features': 2, 'n_estimators': 10}
53099.54207939212 {'max_features': 2, 'n_estimators': 30}
60938.99698679386 {'max_features': 4, 'n_estimators': 3}
52585.36726854069 {'max_features': 4, 'n_estimators': 10}
50684.63919850133 {'max_features': 4, 'n_estimators': 30}
59478.886807279996 {'max_features': 6, 'n_estimators': 3}
52851.064798669955 {'max_features': 6, 'n_estimators': 10}
50402.80562813449 {'max_features': 6, 'n_estimators': 30}
58978.24373669876 {'max_features': 8, 'n_estimators': 3}
52313.635784851656 {'max_features': 8, 'n_estimators': 10}
50081.87192950168 {'max_features': 8, 'n_estimators': 30}
62302.38383726054 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54140.19442744822 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59044.55365118484 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52502.83442828373 {'bootstrap': False, 'max_features': 3, 'n_estimators'

In [177]:
# grid search for Support Vector Machine Regressor model ->
param_grid = [
    {
     'kernel': ['linear'],
     'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]
    },
    {
     'kernel': ['rbf'],
     'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
     'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]
    }
]
svm_reg = SVR()
svm_grid_search = GridSearchCV(estimator = svm_reg,
                               param_grid = param_grid,
                               cv = 5,
                               scoring = "neg_mean_squared_error",
                               n_jobs = -1,
                               verbose = 2
                              )
svm_grid_search.fit(X=housing_prepped, y=housing_labels)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 23.5min finished


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
                                10000.0, 30000.0],
                          'kernel': ['linear']},
                         {'C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0, 1000.0],
                          'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error', verbose=2)

In [178]:
best_mse = svm_grid_search.best_score_
best_rmse = np.sqrt(-best_mse)
best_rmse

70363.84006944533

In [179]:
svm_grid_search.best_estimator_
# C is maxed -> should fine-tune again with extended range

SVR(C=30000.0, kernel='linear')

In [180]:
# Random search with SVR
# (random search tends to find better hyper-parameters than grid search in the same amount of time)
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, expon

params_distribs = {
    'kernel': ['linear', 'rbf'],
    'C': reciprocal(20, 200000),    # reciprocal distrib for unknown hyper-parameter range
    'gamma': expon(scale=1)         # exponential distrib for roughly known range
}
svm_reg = SVR()
svm_random_search = RandomizedSearchCV(estimator = svm_reg,
                                       param_distributions = params_distribs,
                                       scoring = 'neg_mean_squared_error',
                                       n_jobs = -1,
                                       cv = 5,
                                       verbose = 2
                                      )
svm_random_search.fit(X=housing_prepped, y=housing_labels)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.8min finished


RandomizedSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001CF0624D4F0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001CF06187250>,
                                        'kernel': ['linear', 'rbf']},
                   scoring='neg_mean_squared_error', verbose=2)

In [181]:
best_mse = svm_random_search.best_score_
best_rmse = np.sqrt(-best_mse)
best_rmse

57221.11478834314

In [182]:
svm_random_search.best_params_

{'C': 33848.3437841147, 'gamma': 0.3379421632730737, 'kernel': 'rbf'}

In [191]:
################################################################
# Feature importance
################################################################
# use best Random Forest model to get feature importance (not available with SVR)
feature_importances = forest_grid_search.best_estimator_.feature_importances_
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_1hot_attribs = cat_encoder.categories_[0].tolist()
attributes = num_attribs + extra_attribs + cat_1hot_attribs
#dict(zip(feature_importances, attributes))
sorted(zip(feature_importances, attributes), reverse=True)
# drop unimportant features if any (e.g. only one 'ocean_proximity' category is really useful)

[(0.3577453640131463, 'median_income'),
 (0.1536589802745481, 'INLAND'),
 (0.11215522928115201, 'pop_per_hhold'),
 (0.07128099912602268, 'longitude'),
 (0.06572611084928882, 'bedrooms_per_room'),
 (0.06424447898075396, 'latitude'),
 (0.05647895177709007, 'rooms_per_hhold'),
 (0.04264960849183562, 'housing_median_age'),
 (0.01664738453614082, 'total_rooms'),
 (0.015504779169027988, 'population'),
 (0.01501865087728952, 'total_bedrooms'),
 (0.014812134473503248, 'households'),
 (0.00891297084265308, '<1H OCEAN'),
 (0.003061443288374794, 'NEAR OCEAN'),
 (0.0019608936961452053, 'NEAR BAY'),
 (0.0001420203230277317, 'ISLAND')]

In [275]:
##############################################################
## Re-run SVR model with top few most important features
##############################################################
# first add a transformer to pipeline to select top features

# function to get indices to top features ->
# 'np.argpartition(arr,k)' function is used to create a indirect partitioned copy of array 'arr' 
# with its elements rearranged in such a way that the value of the element in k-th position is in 
# the position it would be in sorted 'arr'. All elements smaller than the k-th element are moved 
# before and all equal or greater are moved after. The ordering of the elements in the two 
# partitions is undefined. It returns an array of indices of the same shape as arr.
# 'np.argpartition(np.array(arr), -k)[-k:]' below returns indices of top k elements of arr
def indices_of_top_k(arr, k):
    """ returns indices of top k elements of input array
    """
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

ix = indices_of_top_k(feature_importances, 5)
feature_importances[ix]

array([0.071281  , 0.35774536, 0.11215523, 0.06572611, 0.15365898])

In [271]:
class TopFeatureSelector(BaseEstimator, TransformerMixin):
    """Selects most important features
    # Arguments:
        feature_importances, Numpy array: values showing importance of each feature;
        k, integer: number of features to be selected.
    """
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(arr=self.feature_importances, k=self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]

In [276]:
# create a pipeline that runs full_pipeline and adds feature selection of top k features
k = 5
prep_and_feature_selection_pipeline = Pipeline([
    ('prep', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k))
])

housing_prepped_top_features = prep_and_feature_selection_pipeline.fit_transform(X = housing)
housing_prepped_top_features.shape

(16512, 5)

In [279]:
# create a pipeline that runs full data prep and model prediction with SVR
# (uses best SVR parameters from random_search above)
prep_select_and_predict_pipeline = Pipeline([
    ('prep', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('prediction', SVR(**svm_random_search.best_params_))
])

prep_select_and_predict_pipeline.fit(X=housing, y=housing_labels)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('attribs_adder',
                                                                   CombinedAttributesAdder()),
                                                                  ('std_scalar',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                           

In [280]:
some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]
print("Predictions:\t", prep_select_and_predict_pipeline.predict(some_data))
print("Targets:\t", some_labels.tolist())

Predictions:	 [184976.96963451 338145.80039496 171898.84848753  54997.7446296 ]
Targets:	 [286600.0, 340600.0, 196900.0, 46300.0]


In [286]:
# auto-explore prep option for SVR with grid search
# (uses prep_select_prediction pipeline from above)
param_grid = [
    {
        'prep__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
        'feature_selection__k': list(range(1, len(feature_importances) + 1))
    }
]

svm_grid_search_prep = GridSearchCV(estimator = prep_select_and_predict_pipeline,
                                param_grid = param_grid, 
                                cv = 5,
                                scoring = 'neg_mean_squared_error',
                                verbose = 2,
                                n_jobs = -1
                               )
svm_grid_search_prep.fit(X=housing, y=housing_labels)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 21.3min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('attribs_adder',
                                                                                          CombinedAttributesAdder()),
                                                                                         ('std_scalar',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
              

In [287]:
best_mse = svm_grid_search_prep.best_score_
best_rmse = np.sqrt(-best_mse)
best_rmse

56914.851962975175

In [288]:
svm_grid_search_prep.best_params_
# auto-selects top 10 features

{'feature_selection__k': 10, 'prep__num__imputer__strategy': 'most_frequent'}

In [289]:
svm_grid_search_prep.best_estimator_

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('attribs_adder',
                                                                   CombinedAttributesAdder()),
                                                                  ('std_scalar',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                    

In [291]:
##############################################################
## Evaluate best model on test set
##############################################################
#final_model = forest_grid_search.best_estimator_
final_model = svm_grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

# prep test data with transform() (NOT fit_transform())
X_test_prepped = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepped)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

68139.43891327262

In [292]:
# 95% confidence interval of score
from scipy import stats
confidence = .95
squared_errors = (y_test - final_predictions) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(), 
        scale=stats.sem(squared_errors)))

array([65736.61788422, 70460.36715754])